# Instructions

- Download [this](homework.Rmd) notebook and write your solutions inside it.

- Add the notebook with your solutions to your allocated GitHub repo. (Remember to commit and push.)

- To solve the tasks, finish the code skeletons provided.
  **The code must be runnable to get full points**.
  If you're having trouble, try to explain in your own words 
  what you were trying to do. Proper understanding is half of the points.

- Each task is independent but subtasks are linked.

- To **pass the homework**, you need **3 / 6** points.

- Remember to run the tests for each function you'll be completing.
And feel free to add some of your own.
By testing code in small modular parts (e.g. functions),
you can trust these parts in "downstream" (more complex) applications.

The terms "cost", "distance", and "penalty" are used interchangeably since 
in this context they represent equivalent ways of looking at the same concepts. 
It also helps illustrate that these algorithms are quite general and can be applied to many different problems.

## Advice

* Always restart R (*Session* > *Restart R*) and rerun the notebook
from the top to double-check (i.e. reproduce) your solutions. 
To make sure you get a clean session, prevent RStudio from loading the previous workspace: 
*Tools > Global options*, then in the General tab disable "Restore .RData into workspace at startup",
and for "Save workspace to .RData on exit" select "Never".

* Remember that R starts indexing vectors at position `1`. 
Many other languages in which these algorithms are implemented start at `0`.

* If the code doesn't behave the way you want or you get errors,
try checking out the chapter on [debugging](https://adv-r.hadley.nz/debugging.html) 
from *Advanced R* for different approaches to deal with this.


# Task 1: Global Alignment

Complete the Needleman–Wunsch implementation.

## Part a: The distance between two letters [1 pt]

Fill in the "blanks" in the code below (replace the # WRITE CONDITION HERE) 
with the appropriate logical checks such that this distance function gives
the correct distance between two letters.

Remember to run the code chunk so the notebook starts using your function.

In [1]:

def dist_global(char1, char2):
    #############################
    # Function that computes the distance between two characters in a sequence,
    # used for global alignment.
    # 
    # Input arguments:
    #    char1, char2 = Values of 2 characters, either nucleobases or gaps, 
    #                   as 1-character strings.
    # 
    #    E.g. char1 = 'A', char2 = 'G'
    #         char1 = 'A', char2 = '-'
    # 
    # Returns:
    #    The distance (or cost of change) between char1 and char2, 
    #    using the DIST_... values
    #############################

    DIST_MATCH  = 0
    DIST_GAP = 8
    DIST_TRANSITION = 2
    DIST_TRANSVERSION = 4

    if (char1 == char2):
        return DIST_MATCH 

    if (char1 == '-' or char2 == '-'):# WRITE CONDITION HERE
        return DIST_GAP 

    if ((char1 == 'A' and char2 == 'G') or (char1 == 'G' and char2 == 'A')):# WRITE CONDITION HERE
        return DIST_TRANSITION 

    if  ((char1 == 'T' and char2 == 'C') or (char1 == 'C' and char2 == 'T')):# WRITE CONDITION HERE
        return DIST_TRANSITION

    else:
        return DIST_TRANSVERSION

### Tests

Check your code with the test below, assuming you use the same values as in the lecture.
If you want to experiment with different distance values, 
then naturally the test will have to be adapted.


In [ ]:
print(dist_global('A', 'G') == 2)
print(dist_global('A', '-') == 8)


## Part b: Computing the global alignment [1 pt]

Here you are provided with skeleton code to compute the global alignment
between two sequences using the Needleman–Wunsch algorithm algorithm.
The same notation is used as in the course.

In [14]:
def global_alignment_matrix(seq1, seq2):
    # Convert to character vectors to easily work letter by letter
    # E.g. seq1 = "ABC" becomes ["A", "B", "C"] and seq1[1] == "A"
    seq1 = list(seq1)
    seq2 = list(seq2)

    #D = [[0] * (len(seq2) + 1) for _ in range(len(seq1) + 1)]
    D = np.zeros([len(seq1)+1, len(seq2)+1])

    for i in range(1, len(seq1) + 1):
        D[i][0] = D[i - 1][0] + dist_global(seq1[i - 1], "-")

    for j in range(1, len(seq2) + 1):
        D[0][j] = D[0][j - 1] + dist_global("-", seq2[j - 1])

    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            dist_from_upper_left = dist_global(seq1[i - 1], seq2[j - 1])
            dist_from_above = dist_global(seq1[i - 1], "-")
            dist_from_left = dist_global("-", seq2[j - 1])

            D[i][j] = min(
                D[i - 1][j - 1] + dist_from_upper_left,
                D[i - 1][j] + dist_from_above,
                D[i][j - 1] + dist_from_left
            )

    return D

def global_alignment_value(global_alignment_matrix):
    # Input:
    # global_alignment_matrix = a computed global alignment matrix
    # 
    # Returns:
    # The lower-right corner of the matrix, since we know that holds
    # the global alignment value
    return(global_alignment_matrix[-1,-1])


### Tests

Check your code with these. The last 2 lines compare expected and actual values.

In [21]:
import numpy as np
seq1 = "TACGTCAGC"
seq2 = "TATGTCATGC"

expected_distance = 10

expected_dist_matrix = np.array([
        [ 0,  8, 16, 24, 32, 40, 48, 56, 64, 72, 80],
        [ 8,  0,  8, 16, 24, 32, 40, 48, 56, 64, 72],
        [16,  8,  0,  8, 16, 24, 32, 40, 48, 56, 64],
        [24, 16,  8,  2, 10, 18, 24, 32, 40, 48, 56],
        [32, 24, 16, 10,  2, 10, 18, 26, 34, 40, 48],
        [40, 32, 24, 16, 10,  2, 10, 18, 26, 34, 42],
        [48, 40, 32, 24, 18, 10,  2, 10, 18, 26, 34],
        [56, 48, 40, 32, 26, 18, 10,  2, 10, 18, 26],
        [64, 56, 48, 40, 32, 26, 18, 10,  6, 10, 18],
        [72, 64, 56, 48, 40, 34, 26, 18, 12, 10, 10]
        ])


D = global_alignment_matrix(seq1, seq2)

print(np.all(np.equal(D, expected_dist_matrix)))
print(global_alignment_value(D) == expected_distance)

[[ 0.  8. 16. 24. 32. 40. 48. 56. 64. 72. 80.]
 [ 8.  0.  8. 16. 24. 32. 40. 48. 56. 64. 72.]
 [16.  8.  0.  8. 16. 24. 32. 40. 48. 56. 64.]
 [24. 16.  8.  2. 10. 18. 24. 32. 40. 48. 56.]
 [32. 24. 16. 10.  2. 10. 18. 26. 34. 40. 48.]
 [40. 32. 24. 16. 10.  2. 10. 18. 26. 34. 42.]
 [48. 40. 32. 24. 18. 10.  2. 10. 18. 26. 34.]
 [56. 48. 40. 32. 26. 18. 10.  2. 10. 18. 26.]
 [64. 56. 48. 40. 32. 26. 18. 10.  6. 10. 18.]
 [72. 64. 56. 48. 40. 34. 26. 18. 12. 10. 10.]]
[[ 0  8 16 24 32 40 48 56 64 72 80]
 [ 8  0  8 16 24 32 40 48 56 64 72]
 [16  8  0  8 16 24 32 40 48 56 64]
 [24 16  8  2 10 18 24 32 40 48 56]
 [32 24 16 10  2 10 18 26 34 40 48]
 [40 32 24 16 10  2 10 18 26 34 42]
 [48 40 32 24 18 10  2 10 18 26 34]
 [56 48 40 32 26 18 10  2 10 18 26]
 [64 56 48 40 32 26 18 10  6 10 18]
 [72 64 56 48 40 34 26 18 12 10 10]]
True
True
